# Homework 3 - Attacks and Defenses in Federated Learning

## infrastructure

Based on a variant of FedAvg that updates parameter differences rather than a complete.
As regular FedAvg, this variant performs multiple local epochs.
Unlike regular FedAvg, clients now determine the deviation of their local parameters from the servers' parameters and upload the differences to the server.
Perhaps surprisingly, these differences are referred to as "gradients".

This variant is pretty much equivalent to the original one but makes it easier to reason about updates.

In [ ]:
# Also includes all the dependencies of this module, as the dependencies are members of that module
import typing

from fedavg import *
from matplotlib import pyplot as plt

if torch.accelerator.is_available():
    device = torch.accelerator.current_accelerator()
    print(f"Using accelerator '{device}'")
else:
    device = torch.device("cpu")
    print("WARN: No accelerator found, running on CPU")

In [ ]:
class GradWeightClient(Client):
    def __init__(
        self, client_data: Subset, lr: float, batch_size: int, nr_epochs: int
    ) -> None:
        super().__init__(client_data, batch_size)
        self.optimizer = SGD(params=self.model.parameters(), lr=lr)
        self.nr_epochs = nr_epochs

    def update(self, weights: list[torch.Tensor], seed: int) -> list[torch.Tensor]:
        # Load the server weights into the model
        with torch.no_grad():
            for client_values, server_values in zip(self.model.parameters(), weights):
                client_values[:] = server_values

        torch.manual_seed(seed)

        # Save initial weights to calculate the gradients later
        initial_weights = [param.clone() for param in self.model.parameters()]

        for _epoch in range(self.nr_epochs):
            train_epoch(self.model, self.loader_train, self.optimizer)

        # Calculate gradients: initial_weights - final_weights
        gradients = [
            initial_weight - final_weight
            for final_weight, initial_weight in zip(
                self.model.parameters(), initial_weights
            )
        ]
        return [gradient.detach().cpu().clone() for gradient in gradients]


class FedAvgGradServer(DecentralizedServer, ABC):
    def __init__(
        self,
        lr: float,
        batch_size: int,
        client_subsets: list[Subset],
        client_fraction: float,
        nr_local_epochs: int,
        seed: int,
    ) -> None:
        super().__init__(lr, batch_size, client_subsets, client_fraction, seed)
        self.name = "FedAvg"
        self.nr_local_epochs = nr_local_epochs
        self.clients = [
            GradWeightClient(subset, lr, batch_size, nr_local_epochs)
            for subset in client_subsets
        ]


def is_client_benign(client: Client) -> bool:
    # benign clients are instances of 'GradWeightClient', malicious clients are subtypes
    return type(client) == GradWeightClient

In [ ]:
type SplitDataset = typing.Sequence[torch.utils.data.Dataset[typing.Any]]

### attacks

In [ ]:
class AttackerGradientReversion(GradWeightClient):
    def update(self, weights: list[torch.Tensor], seed: int) -> list[torch.Tensor]:
        # Call the base implementation to perform the normal training and update process
        honest_gradients = super().update(weights, seed)

        # Reverse the gradients: for a malicious client, negate the weights to simulate an attack
        reversed_gradients = [-5*w for w in honest_gradients]
    

        return reversed_gradients

def build_gradient_reversion_client(
    client_data: torch.utils.data.Dataset[typing.Any],
    learning_rate: float,
    batch_size: int,
    epochs: int,
):
    return AttackerGradientReversion(client_data, learning_rate, batch_size, epochs)

In [ ]:
class AttackerPartGradientReversion(GradWeightClient):
    def update(self, weights: list[torch.Tensor], seed: int) -> list[torch.Tensor]:
        # Set random seed for reproducibility
        torch.manual_seed(seed)

        # Call the base implementation to perform the normal training and update process
        honest_gradients = super().update(weights, seed)

        # Flatten the gradients and calculate 10% of the total number of gradient elements
        all_gradients = torch.cat([g.flatten() for g in honest_gradients])
        num_to_manipulate = int(all_gradients.numel())

        # Determine the threshold of parameters to modify
        param_threshold = (
            num_to_manipulate * 0.00001
        )  # Set this to your desired threshold

        # Select the first k layers until the cumulative number of parameters exceeds the threshold
        cumulative_params = 0
        selected_gradients = []
        for gradient in honest_gradients:
            # print(gradient.numel())
            cumulative_params += gradient.numel()
            selected_gradients.append(gradient)
            if cumulative_params >= param_threshold:
                break

        # Multiply the selected gradients by -1000
        for gradient in selected_gradients:
            gradient *= -1000

        return honest_gradients  # Return the modified gradients


def build_partial_gradient_reversion_client(
    client_data: torch.utils.data.Dataset[typing.Any],
    learning_rate: float,
    batch_size: int,
    epochs: int,
):
    return AttackerPartGradientReversion(client_data, learning_rate, batch_size, epochs)

In [ ]:
# a new data class that help us to process batch
@dataclass
class Batch:
    batch_id: int
    inputs: torch.Tensor
    labels: torch.Tensor

    def __post_init__(self):
        self.batch_size = self.inputs.shape[0]

    def to(self, device):
        inputs = self.inputs.to(device)
        labels = self.labels.to(device)
        return Batch(self.batch_id, inputs, labels)

    def clone(self):
        inputs = self.inputs.clone()
        labels = self.labels.clone()
        return Batch(self.batch_id, inputs, labels)

    def clip(self, batch_size):
        if batch_size is None:
            return self

        inputs = self.inputs[:batch_size]
        labels = self.labels[:batch_size]

        return Batch(self.batch_id, inputs, labels)


class Synthesizer:
    def __init__(self, poisoning_proportion):
        self.poisoning_proportion = poisoning_proportion

    def make_backdoor_batch(self, batch: Batch, test=False, attack=True) -> Batch:
        if not attack:
            return batch

        if test:
            attack_portion = batch.batch_size
        else:
            attack_portion = round(batch.batch_size * self.poisoning_proportion)

        backdoored_batch = batch.clone()
        self.apply_backdoor(backdoored_batch, attack_portion)
        return backdoored_batch

    def apply_backdoor(self, batch, attack_portion):
        """
        Modifies only a portion of the batch (represents batch poisoning).

        :param batch:
        :return:
        """
        self.synthesize_inputs(batch=batch, attack_portion=attack_portion)
        self.synthesize_labels(batch=batch, attack_portion=attack_portion)

        return

    def synthesize_inputs(self, batch, attack_portion=None) -> None:
        raise NotImplemented

    def synthesize_labels(self, batch, attack_portion=None) -> None:
        raise NotImplemented


class PatternSynthesizer(Synthesizer):
    pattern_tensor: torch.Tensor = torch.tensor(
        [
            [1.0, 0.0, 1.0],
            [-10.0, 1.0, -10.0],
            [-10.0, -10.0, 0.0],
            [-10.0, 1.0, -10.0],
            [1.0, 0.0, 1.0],
        ]
    )
    "Just some random 2D pattern."

    x_top = 3
    "X coordinate to put the backdoor into."
    y_top = 23
    "Y coordinate to put the backdoor into."

    mask_value = -10
    "A tensor coordinate with this value won't be applied to the image."

    resize_scale = (5, 10)
    "If the pattern is dynamically placed, resize the pattern."

    mask: torch.Tensor = None
    "A mask used to combine backdoor pattern with the original image."

    pattern: torch.Tensor = None
    "A tensor of the `input.shape` filled with `mask_value` except backdoor."

    def __init__(self, poisoning_proportion):
        super().__init__(poisoning_proportion)
        self.input_shape = (1, 28, 28)
        self.backdoor_label = 0
        self.normalize = transforms.Normalize((0.1307,), (0.3081,))
        self.make_pattern(self.pattern_tensor, self.x_top, self.y_top)

    def make_pattern(self, pattern_tensor, x_top, y_top):
        # put pattern into the image
        full_image = torch.zeros(self.input_shape)
        full_image.fill_(self.mask_value)
        # full image has a pixel value of -10
        x_bot = x_top + pattern_tensor.shape[0]
        y_bot = y_top + pattern_tensor.shape[1]

        if x_bot >= self.input_shape[1] or y_bot >= self.input_shape[2]:
            raise ValueError(
                f"Position of backdoor outside image limits:"
                f"image: {self.input_shape}, but backdoor"
                f"ends at ({x_bot}, {y_bot})"
            )

        full_image[:, x_top:x_bot, y_top:y_bot] = pattern_tensor
        # full image has a pixel value of -10 except for the backdoor (pattern_tensor) size: 5 * 3
        self.mask = 1 * (full_image != self.mask_value).to(device)  # (0, 1)
        # mask is a tensor of 0 and 1, 0 for -10 and 1 for other values
        self.pattern = self.normalize(full_image).to(device)  # )(-52.5678, 2.7537)

    def synthesize_inputs(self, batch, attack_portion=None):
        pattern, mask = self.get_pattern()
        # mask value (0, 1); value 0, keep the original image; value 1, replace with pattern
        batch.inputs[:attack_portion] = (1 - mask) * batch.inputs[
            :attack_portion
        ] + mask * pattern

        return

    def synthesize_labels(self, batch, attack_portion=None):
        batch.labels[:attack_portion].fill_(self.backdoor_label)
        return

    def get_pattern(self):
        return self.pattern, self.mask


poisoning_proportion = 0.5
synthesizer = PatternSynthesizer(poisoning_proportion=poisoning_proportion)


class AttackerBackdoor(GradWeightClient):
    def train_epoch(
        self, model: torch.nn.Module, loader: DataLoader, optimizer: Optimizer
    ) -> None:
        model.train()
        for batch_idx, (inputs, labels) in enumerate(loader):
            batch = Batch(batch_idx, inputs, labels)
            batch = batch.to(device)
            backdoored_batch = synthesizer.make_backdoor_batch(
                batch, test=False, attack=True
            )
            data = backdoored_batch.inputs
            target = backdoored_batch.labels
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

    def update(self, weights: list[torch.Tensor], seed: int) -> list[torch.Tensor]:
        # Load the server weights into the model
        with torch.no_grad():
            for client_values, server_values in zip(self.model.parameters(), weights):
                client_values[:] = server_values

        torch.manual_seed(seed)

        # Save initial weights to calculate the gradients later
        initial_weights = [param.clone() for param in self.model.parameters()]

        for _epoch in range(self.nr_epochs):
            self.train_epoch(self.model, self.loader_train, self.optimizer)

        # Calculate gradients: final_weights - initial_weights
        gradients = [
            5 * (initial_weight - final_weight)
            for final_weight, initial_weight in zip(
                self.model.parameters(), initial_weights
            )
        ]

        return [gradient.detach().cpu().clone() for gradient in gradients]


def build_backdoor_client(
    client_data: torch.utils.data.Dataset[typing.Any],
    learning_rate: float,
    batch_size: int,
    epochs: int,
):
    return AttackerBackdoor(
        client_data=client_data,
        lr=learning_rate,
        batch_size=batch_size,
        nr_epochs=epochs,
    )

### defenses

In [ ]:
def krum(
    clients_updates: list[tuple[int, torch.Tensor]],
    total_update_count: int = 20,
    max_malicious_update_count: int = 4,
) -> list[int]:
    nearest_neighbors_count = total_update_count - max_malicious_update_count - 2
    scores = []

    # Calculate the score for each update
    for i in range(len(clients_updates)):
        distances = []
        for j in range(len(clients_updates)):
            if i != j:
                # Compute squared l2 distance between updates
                distance = sum(
                    (param_i - param_j).pow(2).sum()
                    for param_i, param_j in zip(
                        clients_updates[i][1], clients_updates[j][1]
                    )
                )
                distances.append(distance)

        nearest_distances = sorted(distances)[:nearest_neighbors_count]

        # Sum of these distances is the score of this client's update
        scores.append(sum(nearest_distances))

    # Select the index with the smallest score
    selected_index = scores.index(min(scores))
    return [selected_index]

In [ ]:
def multi_krum(
    clients_updates: list[tuple[int, torch.Tensor]],
    considered_update_count=14,
    total_update_count: int = 20,
    max_malicious_update_count: int = 5,
) -> list[int]:
    selected_indices = []
    remaining_updates = clients_updates[:]

    for i in range(considered_update_count):
        # Apply Krum to the remaining updates and get the index of the best gradient
        selected_index = krum(
            remaining_updates, total_update_count - i, max_malicious_update_count
        )[0]

        # Map selected index from remaining_updates back to the original clients_updates
        original_index = clients_updates.index(remaining_updates[selected_index])
        selected_indices.append(original_index)

        # Remove the selected update from remaining_updates
        remaining_updates.pop(selected_index)

    return selected_indices

In [ ]:
# contains the gradients for all parameters from one client
type ClientUpdate = list[torch.Tensor]


def trimmed_mean(all_updates: list[ClientUpdate], beta: float = 0.4):
    # 'n_attackers' is the number of top and bottom values to exclude from the mean calculation.

    # Transpose the list to group gradients of the same parameter together
    parameter_wise_updates = list(zip(*all_updates))
    n_attackers = int(20 * beta)
    trimmed_mean_gradients = []
    for parameter_group in parameter_wise_updates:
        # Stack gradients to create a new dimension corresponding to clients
        stacked_updates = torch.stack(parameter_group, dim=0)

        # Sort updates along the client dimension
        sorted_updates = torch.sort(stacked_updates, dim=0)[0]
        # Compute mean excluding the top and bottom 'n_attackers' updates
        if n_attackers > 0:
            trimmed_updates = sorted_updates[n_attackers:-n_attackers]
        else:
            trimmed_updates = sorted_updates

        mean_gradient = torch.mean(trimmed_updates, dim=0)
        trimmed_mean_gradients.append(mean_gradient * 20)

    return trimmed_mean_gradients

In [ ]:
def majority_sign_filter(all_updates: list[ClientUpdate]) -> list[torch.Tensor]:
    # Step 1: Transpose the list to group gradients of the same parameter together
    parameter_wise_updates = list(zip(*all_updates))

    # Step 2: Calculate sign and count for determining majority
    all_signs = [
        torch.sign(torch.stack(parameter_group, dim=0))
        for parameter_group in parameter_wise_updates
    ]

    # Step 3: Determine the majority sign for each parameter coordinate
    majority_signs = [torch.sign(torch.sum(signs, dim=0)) for signs in all_signs]

    # Step 4: Filter out gradients whose signs don't match the majority
    filtered_updates = []
    for idx, parameter_group in enumerate(parameter_wise_updates):
        majority_sign = majority_signs[idx]
        zeroed_gradients = []

        for gradient in parameter_group:
            gradient_signs = torch.sign(gradient)
            # Zero out gradients where the sign does not match the majority
            gradient[gradient_signs != majority_sign] = 0
            zeroed_gradients.append(gradient)

        # Step 5: Calculate the average of the filtered gradients
        filtered_average = torch.mean(torch.stack(zeroed_gradients), dim=0)
        filtered_updates.append(filtered_average * 20)

    return filtered_updates

In [ ]:
def clipping(
    all_updates: list[ClientUpdate],
    clip_norm_ratio: float = 1.0,
    noise_std_dev: float = 0.01,
) -> list[torch.Tensor]:
    # 'all_updates' is a list of lists of torch.Tensor objects,
    # where each inner list contains the gradients for all parameters from one client.

    # Step 1: Calculate the norm of each client's update
    norms = [
        torch.norm(torch.stack([torch.norm(p) for p in client_update]))
        for client_update in all_updates
    ]
    average_norm = sum(norms) / len(norms) * clip_norm_ratio

    # Step 2: Clip each client's update
    clipped_updates = []
    for client_update in all_updates:
        clipped_update = []
        client_norm = torch.norm(torch.stack([torch.norm(p) for p in client_update]))
        clip_scale = min(
            1, average_norm / (client_norm + 1e-6)
        )  # Avoid division by zero

        for gradient in client_update:
            clipped_gradient = gradient * clip_scale

            clipped_update.append(clipped_gradient)

        clipped_updates.append(clipped_update)

    # Step 4: Compute the average of these modified gradients
    parameter_wise_updates = list(zip(*clipped_updates))

    # TODO: Wouldn't this be equivalent to summing up?
    averaged_gradients = [
        torch.mean(torch.stack(parameter_group), dim=0) * 20
        for parameter_group in parameter_wise_updates
    ]

    return averaged_gradients

### evaluation

In [ ]:
@dataclass
class BackdoorEvaluationResult:
    accuracy: float
    success_rate: float
    confusion_matrix: np.ndarray


def evaluate_backdoor(
    model, test_loader: DataLoader, classes_count: int = 10
) -> BackdoorEvaluationResult:
    dataset_size = 0
    correct = 0
    successful_attacks = 0

    confusion_matrix = np.zeros((classes_count, classes_count), dtype=np.int16)

    model.eval()
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            batch = Batch(batch_idx, inputs, labels)
            batch = batch.to(device)
            backdoored_batch = synthesizer.make_backdoor_batch(
                batch, test=True, attack=True
            )

            batch_data = backdoored_batch.inputs
            batch_target = backdoored_batch.labels

            batch_output = model(batch_data)
            batch_predictions = batch_output.argmax(dim=1, keepdim=True)

            dataset_size += batch_data.shape[0]
            correct += (
                batch_predictions.eq(batch.labels.view_as(batch_predictions))
                .sum()
                .item()
            )
            successful_attacks += (
                batch_predictions.eq(batch_target.view_as(batch_predictions))
                .sum()
                .item()
            )

            # Update confusion matrix
            for target_value, predicted_value in zip(
                batch.labels.view(-1), batch_predictions.view(-1)
            ):
                confusion_matrix[target_value.long(), predicted_value.long()] += 1

    accuracy = 100.0 * correct / dataset_size
    success_rate = 100.0 * successful_attacks / dataset_size

    return BackdoorEvaluationResult(
        accuracy=accuracy, success_rate=success_rate, confusion_matrix=confusion_matrix
    )

In [ ]:
import seaborn as sns


def plot_confusion_matrix(confusion_matrix: np.ndarray):
    class_names = [str(it) for it in range(10)]

    plt.figure(figsize=(10, 7))
    sns.heatmap(
        confusion_matrix,
        annot=True,
        fmt="d",
        cmap="Blues",
        xticklabels=class_names,
        yticklabels=class_names,
    )
    plt.xlabel("Predicted labels")
    plt.ylabel("True labels")
    plt.title("Confusion Matrix")
    plt.show()

## Question 1: Influence of Non-I.I.D. Data in Attacks and Defenses (4 points)

Using the template code from the overview notebook ([src](./overview.ipynb)), experiment with how non-i.i.d. data distributions influence the performance of attacks and defenses. Utilize the `AttackerGradientReversion` for the attack. Keep all hyperparameters the same as those in the template code, except modify the data distribution among the clients to be non-i.i.d.

### Scoring:

- _(1 point)_ Generate test accuracy for 10 rounds, both when there is no defense and when the defenses are `krum`, `multi_krum`, and `majority_sign`. 
- _(1 point)_ Compile the results of these runs into one plot to visually compare the influence of each defense mechanism under non-i.i.d. conditions.
- _(2 points)_ Discuss the differences (if any) between the results in i.i.d. and non-i.i.d. settings for each defense mechanism. 


### implementation

In [ ]:
class FedAvgServerDefense(FedAvgGradServer):
    def __init__(
        self,
        lr: float,
        batch_size: int,
        client_subsets: list,
        client_fraction: float,
        nr_local_epochs: int,
        seed: int,
        defense=None,
    ):
        super().__init__(
            lr, batch_size, client_subsets, client_fraction, nr_local_epochs, seed
        )
        self.defense_method = defense

    def run(self, nr_rounds: int) -> RunResult:
        # The beginning of the method remains unchanged
        elapsed_time = 0.0
        run_result = RunResult(
            self.name,
            self.nr_clients,
            self.client_fraction,
            self.batch_size,
            self.nr_local_epochs,
            self.lr,
            self.seed,
        )

        # Initial server weights
        server_weights = [param.data.clone() for param in self.model.parameters()]
        for nr_round in tqdm(range(nr_rounds), desc="Rounds", leave=False):
            setup_start_time = perf_counter()
            self.model.train()
            weights = [x.detach().cpu().clone() for x in self.model.parameters()]
            indices_chosen_clients = self.rng.choice(
                self.nr_clients, self.nr_clients_per_round, replace=False
            )
            chosen_sum_nr_samples = sum(
                self.client_sample_counts[i] for i in indices_chosen_clients
            )
            chosen_adjusted_weights: list[list[torch.Tensor]] = []
            elapsed_time += perf_counter() - setup_start_time
            update_time = 0.0

            client_updates = []  # Store (client_index, updates) tuples
            for c_i in indices_chosen_clients:
                update_start_time = perf_counter()
                ind = int(c_i)
                client_round_seed = (
                    self.seed + ind + 1 + nr_round * self.nr_clients_per_round
                )
                client_weights = self.clients[ind].update(weights, client_round_seed)
                client_updates.append((ind, client_weights))
                update_time = max(update_time, perf_counter() - update_start_time)
            if self.defense_method:
                selected_indices = self.defense_method(client_updates)
            else:
                selected_indices = range(
                    len(client_updates)
                )  # Use all updates if no defense is specified

            self._print_selected_clients(indices_chosen_clients[selected_indices].tolist())

            chosen_sum_nr_samples = sum(
                self.client_sample_counts[i]
                for i in indices_chosen_clients[selected_indices]
            )
            chosen_adjusted_weights = [
                [
                    self.client_sample_counts[indices_chosen_clients[ind]]
                    / chosen_sum_nr_samples
                    * tens
                    for tens in client_updates[ind][1]
                ]
                for ind in selected_indices
            ]
            # Aggregation logic remains the same from this point onwards
            # Note that chosen_adjusted_weights now only contains updates from clients selected by multi_krum
            elapsed_time += update_time
            aggregate_start_time = perf_counter()
            averaged_gradients: list[torch.Tensor] = [
                sum(x) for x in zip(*chosen_adjusted_weights)
            ]

            # Update server model with averaged gradients
            with torch.no_grad():
                for server_weight, gradient in zip(server_weights, averaged_gradients):
                    server_weight -= gradient.to(
                        device=device
                    )  # Assume learning rate is absorbed in gradient

            # Distribute updated weights to clients
            with torch.no_grad():
                for param, server_weight in zip(
                    self.model.parameters(), server_weights
                ):
                    param.data[:] = server_weight.data

            elapsed_time += perf_counter() - aggregate_start_time
            run_result.wall_time.append(round(elapsed_time, 1))
            run_result.message_count.append(
                2 * (nr_round + 1) * self.nr_clients_per_round
            )
            run_result.test_accuracy.append(self.test())

        return run_result

    def _print_selected_clients(self, client_indices: typing.Iterable[int]) -> None:
        ordered_indices = list(sorted(client_indices))
        malicious_indices = list(
            filter(lambda idx: not is_client_benign(self.clients[idx]), ordered_indices)
        )

        print(f"selected clients: {ordered_indices} (whereof {len(malicious_indices) } are malicious)")

In [ ]:
class FedAvgServerDefenseCoordinate(FedAvgGradServer):
    def __init__(
        self,
        lr: float,
        batch_size: int,
        client_subsets: list,
        client_fraction: float,
        nr_local_epochs: int,
        seed: int,
        defense=None,
    ):
        super().__init__(
            lr, batch_size, client_subsets, client_fraction, nr_local_epochs, seed
        )
        self.defense_method = defense

    def run(self, nr_rounds: int) -> RunResult:
        # The beginning of the method remains unchanged
        elapsed_time = 0.0
        run_result = RunResult(
            self.name,
            self.nr_clients,
            self.client_fraction,
            self.batch_size,
            self.nr_local_epochs,
            self.lr,
            self.seed,
        )

        # Initial server weights
        server_weights = [param.data.clone() for param in self.model.parameters()]
        for nr_round in tqdm(range(nr_rounds), desc="Rounds", leave=False):
            setup_start_time = perf_counter()
            self.model.train()
            weights = [x.detach().cpu().clone() for x in self.model.parameters()]
            indices_chosen_clients = self.rng.choice(
                self.nr_clients, self.nr_clients_per_round, replace=False
            )
            chosen_sum_nr_samples = sum(
                self.client_sample_counts[i] for i in indices_chosen_clients
            )
            chosen_adjusted_weights: list[list[torch.Tensor]] = []
            elapsed_time += perf_counter() - setup_start_time
            update_time = 0.0

            for c_i in indices_chosen_clients:
                update_start_time = perf_counter()
                ind = int(c_i)
                client_round_seed = (
                    self.seed + ind + 1 + nr_round * self.nr_clients_per_round
                )
                client_weights = self.clients[ind].update(weights, client_round_seed)
                chosen_adjusted_weights.append(
                    [
                        self.client_sample_counts[ind] / chosen_sum_nr_samples * tens
                        for tens in client_weights
                    ]
                )
                update_time = max(update_time, perf_counter() - update_start_time)

            elapsed_time += update_time
            aggregate_start_time = perf_counter()
            if self.defense_method:
                averaged_gradients = self.defense_method(chosen_adjusted_weights)
            else:
                averaged_gradients: list[torch.Tensor] = [
                    sum(x) for x in zip(*chosen_adjusted_weights)
                ]

            # Update server model with averaged gradients
            with torch.no_grad():
                for server_weight, gradient in zip(server_weights, averaged_gradients):
                    server_weight -= gradient.to(
                        device=device
                    )  # Assume learning rate is absorbed in gradient

            # Distribute updated weights to clients
            with torch.no_grad():
                for param, server_weight in zip(
                    self.model.parameters(), server_weights
                ):
                    param.data[:] = server_weight.data

            elapsed_time += perf_counter() - aggregate_start_time
            run_result.wall_time.append(round(elapsed_time, 1))
            run_result.message_count.append(
                2 * (nr_round + 1) * self.nr_clients_per_round
            )
            run_result.test_accuracy.append(self.test())

        return run_result

### establish baseline

In [ ]:
dataset = split(nr_clients=100, iid=True, seed=42)
FedAvgServerDefense(0.02, 200, dataset, 0.2, 2, 42, None).run(10)

### benchmarks

In [ ]:
type _ClientFactory = typing.Callable[
    [torch.utils.data.Dataset[typing.Any], float, int, int], Client
]


def inject_malicious_clients(
    server: Server,
    malicious_client_factory: _ClientFactory,
):
    clients = server.clients

    # Number of clients to be made malicious with gradient reversion
    num_malicious = int(0.20 * len(clients))  # For example, 20%

    # Randomly select indices for the clients to undergo gradient inversion attack
    generator = np.random.default_rng(43)
    malicious_indices = generator.choice(len(clients), num_malicious, replace=False)

    # Replace the selected clients with instances of AttackerGradientInversion
    print(f"injected malicious clients at indices: {np.sort(malicious_indices)}")

    for idx in malicious_indices:
        client_data = clients[idx].loader_train.dataset
        lr = clients[idx].optimizer.param_groups[0]["lr"]
        batch_size = clients[idx].loader_train.batch_size
        nr_epochs = clients[idx].nr_epochs

        clients[idx] = malicious_client_factory(client_data, lr, batch_size, nr_epochs)


def configure_server_index(dataset: SplitDataset, defense: typing.Any) -> Server:
    server = FedAvgServerDefense(
        lr=0.02,
        batch_size=200,
        client_subsets=dataset,
        client_fraction=0.2,
        nr_local_epochs=2,
        seed=42,
        defense=defense,
    )

    inject_malicious_clients(
        server, malicious_client_factory=build_gradient_reversion_client
    )
    return server


def configure_server_parameter(dataset: SplitDataset, defense: typing.Any) -> Server:
    server = FedAvgServerDefenseCoordinate(
        lr=0.02,
        batch_size=200,
        client_subsets=dataset,
        client_fraction=0.2,
        nr_local_epochs=2,
        seed=42,
        defense=defense,
    )

    inject_malicious_clients(
        server, malicious_client_factory=build_gradient_reversion_client
    )
    return server


def run_training(
    iid: bool, method: typing.Literal["krum", "multi-krum", "majority-sign"] | None
) -> RunResult:
    dataset = split(nr_clients=100, iid=iid, seed=42)

    match method:
        case "krum":
            return configure_server_index(dataset=dataset, defense=krum).run(10)
        case "multi-krum":
            return configure_server_index(dataset=dataset, defense=multi_krum).run(10)
        case "majority-sign":
            return configure_server_parameter(dataset=dataset, defense=majority_sign_filter).run(10)
        case None:
            return configure_server_index(dataset=dataset, defense=None).run(10)
            # 77.2, 81.01, 83.13, 85.26, 85.03, 87.22])
            #return FedAvgGradServer(0.02, 200, sample_split, 0.2, 2, 42).run(10)

In [ ]:
defense_mechanism_variants = ["krum", "multi-krum", "majority-sign", None]

In [ ]:
results_defense_mechanism_iid = [
    run_training(iid=True, method=method)
    for method in tqdm(defense_mechanism_variants, "defense mechanism", leave=False)
]

In [ ]:
results_defense_mechanism_non_iid = [
    run_training(iid=False, method=method)
    for method in tqdm(defense_mechanism_variants, "defense mechanism", leave=False)
]

In [ ]:
results_defense_mechanism_iid[2].as_df()

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("test accuracy")
for statistics, variant_name in zip(results_defense_mechanism_iid, defense_mechanism_variants):
    plt.plot(statistics.test_accuracy, label=f"{variant_name}", alpha=0.7)
plt.legend()
# plt.ylim(None, .3)

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("test accuracy")
for statistics, variant_name in zip(results_defense_mechanism_non_iid, defense_mechanism_variants):
    plt.plot(statistics.test_accuracy, label=f"{variant_name}", alpha=0.7)
plt.legend()
# plt.ylim(None, .3)

## Question 2: Implementing Bulyan Defense (5 points)

[Bulyan defense](https://arxiv.org/abs/1802.07927) combines aspects of `multi-krum` and `trimmed-mean` to protect against sophisticated attacks in federated environments. In this question, you will implement Bulyan by modifying and combining the existing code in the [attacks_and_defenses.ipynb](https://github.com/lydiaYchen/DDL25Spring/blob/main/lab/tutorial_3/attacks_and_defenses.ipynb).

### Scoring:
- _(1 point)_ Modify the server-side code to integrate a two-step defense process where:
  1. A client-wise defense like `multi-krum` filters out suspicious gradients based on their distance metrics.
  2. A coordinate-wise defense like `trimmed-mean` calculates the mean across the remaining gradients after trimming the extremes.
  
- _(1 point)_ Successfully integrate these two defense mechanisms [`multi-krum`, `trimmed-mean`] to implement the Bulyan.
- _(3 points)_ Evaluate the performance of your implemented Bulyan defense against three types of attacks: `AttackerGradientReversion`, `AttackerPartGradientReversion`, and `AttackerBackdoor`. Vary the defense hyperparameters to make your defense defend against these attacks while keeping the data distribution i.i.d. and the attacker's hyperparameters constant as per the template. Document the performance of your defense strategy against these attacks in your notebook.

In [ ]:
class CombinedServerDefense(FedAvgGradServer):
    def __init__(
        self,
        lr: float,
        batch_size: int,
        client_subsets: list,
        client_fraction: float,
        nr_local_epochs: int,
        seed: int,
        defense_client_wise=None,
        defense_coordinate_wise=None,
    ):
        super().__init__(
            lr, batch_size, client_subsets, client_fraction, nr_local_epochs, seed
        )
        self._defense_client_wise = defense_client_wise
        self._defense_coordinate_wise = defense_coordinate_wise

    def _sample_clients(self):
        return self.rng.choice(
            self.nr_clients, self.nr_clients_per_round, replace=False
        )

    def _print_selected_clients(self, client_indices: typing.Iterable[int]) -> None:
        ordered_indices = list(sorted(client_indices))
        malicious_indices = list(
            filter(lambda idx: not is_client_benign(self.clients[idx]), ordered_indices)
        )

        print(
            f"selected clients: {ordered_indices} (whereof {len(malicious_indices) } are malicious)"
        )

    def run(self, nr_rounds: int) -> RunResult:
        # The beginning of the method remains unchanged
        elapsed_time = 0.0
        run_result = RunResult(
            self.name,
            self.nr_clients,
            self.client_fraction,
            self.batch_size,
            self.nr_local_epochs,
            self.lr,
            self.seed,
        )

        # Initial server weights
        server_weights = [param.data.clone() for param in self.model.parameters()]
        for nr_round in tqdm(range(nr_rounds), desc="Rounds", leave=False):
            setup_start_time = perf_counter()
            self.model.train()
            weights = [x.detach().cpu().clone() for x in self.model.parameters()]
            indices_chosen_clients = self._sample_clients()
            elapsed_time += perf_counter() - setup_start_time

            update_time = 0.0
            client_updates: list[tuple[int, ClientUpdate]] = []
            for client_idx in indices_chosen_clients.tolist():
                update_start_time = perf_counter()

                client_round_seed = (
                    self.seed + client_idx + 1 + nr_round * self.nr_clients_per_round
                )
                client_weights = self.clients[client_idx].update(
                    weights, client_round_seed
                )
                client_updates.append((client_idx, client_weights))

                update_time = max(update_time, perf_counter() - update_start_time)

            if self._defense_client_wise is not None:
                selected_indices = self._defense_client_wise(client_updates)
            else:
                # Use all updates if no defense is specified
                selected_indices = range(len(client_updates))

            self._print_selected_clients(indices_chosen_clients[selected_indices].tolist())

            chosen_sum_nr_samples = sum(
                self.client_sample_counts[i]
                for i in indices_chosen_clients[selected_indices]
            )
            chosen_adjusted_weights: list[list[torch.Tensor]] = [
                [
                    self.client_sample_counts[indices_chosen_clients[ind]]
                    / chosen_sum_nr_samples
                    * tens
                    for tens in client_updates[ind][1]
                ]
                for ind in selected_indices
            ]

            averaged_gradients: list[torch.Tensor]
            if self._defense_coordinate_wise is not None:
                averaged_gradients = self._defense_coordinate_wise(chosen_adjusted_weights)
            else:
                # Use torch function for summing up
                averaged_gradients = [sum(x) for x in zip(*chosen_adjusted_weights)]

            # Aggregation logic remains the same from this point onwards
            # Note that chosen_adjusted_weights now only contains updates from clients selected by multi_krum
            elapsed_time += update_time
            aggregate_start_time = perf_counter()

            # Update server model with averaged gradients
            with torch.no_grad():
                for server_weight, gradient in zip(server_weights, averaged_gradients):
                    server_weight -= gradient.to(
                        device=device
                    )  # Assume learning rate is absorbed in gradient

            # Distribute updated weights to clients
            with torch.no_grad():
                for param, server_weight in zip(
                    self.model.parameters(), server_weights
                ):
                    param.data[:] = server_weight.data

            elapsed_time += perf_counter() - aggregate_start_time
            run_result.wall_time.append(round(elapsed_time, 1))
            run_result.message_count.append(
                2 * (nr_round + 1) * self.nr_clients_per_round
            )
            run_result.test_accuracy.append(self.test())

        return run_result

In [ ]:
def configure_server(
    dataset: SplitDataset,
    defense_client_wise: typing.Any,
    defense_coordinate_wise: typing.Any,
) -> Server:
    server = CombinedServerDefense(
        lr=0.02,
        batch_size=200,
        client_subsets=dataset,
        client_fraction=0.2,
        nr_local_epochs=2,
        seed=42,
        defense_client_wise=defense_client_wise,
        defense_coordinate_wise=defense_coordinate_wise,
    )

    return server

In [ ]:
@dataclass
class BulyanParameters:
    krum_considered_update_count: int
    krum_max_malicious_update_count: int
    mean_beta: float


def run_training(
    attack_type: typing.Literal[
        "gradient-reversion", "partial-gradient-reversion", "backdoor"
    ],
    parameters: BulyanParameters,
) -> tuple[RunResult, BackdoorEvaluationResult]:
    dataset = split(nr_clients=100, iid=True, seed=42)

    server = configure_server(
        dataset,
        lambda client_updates: multi_krum(
            client_updates,
            considered_update_count=parameters.krum_considered_update_count,
            total_update_count=20,
            max_malicious_update_count=parameters.krum_max_malicious_update_count,
        ),
        lambda all_updates: trimmed_mean(all_updates, beta=parameters.mean_beta),
    )

    match attack_type:
        case "gradient-reversion":
            inject_malicious_clients(
                server, malicious_client_factory=build_gradient_reversion_client
            )
        case "partial-gradient-reversion":
            inject_malicious_clients(
                server, malicious_client_factory=build_partial_gradient_reversion_client
            )
        case 'backdoor':
            inject_malicious_clients(server, malicious_client_factory=build_backdoor_client)

    return server.run(10), evaluate_backdoor(server.model, test_loader, 10)

In [ ]:
variants_bulyan = [
    BulyanParameters(14, 5, 0.1),
    BulyanParameters(14, 5, 0.2),
    BulyanParameters(14, 5, 0.4),
    BulyanParameters(14, 5, 0.6),
    BulyanParameters(7, 5, 0.1),
    BulyanParameters(7, 5, 0.2),
    BulyanParameters(7, 5, 0.4),
    BulyanParameters(7, 5, 0.6),
    BulyanParameters(14, 2, 0.2),
    BulyanParameters(14, 2, 0.4),
]

In [ ]:
results_bulyan: list[RunResult] = []
results_backdoor_bulyan: list[BackdoorEvaluationResult] = []

for variant in tqdm(variants_bulyan, "variant", leave=False):
    result, backdoor_result = run_training('backdoor', variant)

    results_bulyan.append(result)
    results_backdoor_bulyan.append(backdoor_result)

In [ ]:
results_bulyan_reversion: list[RunResult] = []

for variant in tqdm(variants_bulyan, "variant", leave=False):
    result, _ = run_training('gradient-reversion', variant)

    results_bulyan_reversion.append(result)

In [ ]:
results_bulyan_partial_reversion: list[RunResult] = []

for variant in tqdm(variants_bulyan, "variant", leave=False):
    result, _ = run_training('partial-gradient-reversion', variant)

    results_bulyan_partial_reversion.append(result)

## Question 3: Implementing SparseFed Defense (7 points)

In this exercise, you will implement the [SparseFed Defense](https://arxiv.org/abs/2112.06274). SparseFed Defense involves two main steps for each participating device during the training process:
1. **Clipping the gradient:** Each device clips its gradient to reduce the impact of potentially malicious updates.
2. **Aggregating top-k updates:** During the aggregation phase, only the gradients with the highest magnitudes (top-k) are considered. This method is designed to minimize the risk of model poisoning by excluding anomalous gradient contributions from potentially compromised devices.

### Scoring:
- _(2 points)_ Implement a simplified version of SparseFed based on the clipping code provided in the template notebook. Unlike the original algorithm which includes momentum, you only need to implement the clipping step and the sparsification step. However, ensure that only the top-k coordinates in the aggregated gradients are selected based on their magnitudes. For example, if the aggregated gradient is `[3, -1, -1, -3]`, selecting the top-2 by magnitude should result in `[3, 0, 0, -3]`.

- _(3 points)_ Evaluate the performance of your implemented SparseFed defense against two types of attacks: `AttackerGradientReversion` and `AttackerBackdoor`. Adjust the values of the defense hyperparameter k to optimize defense effectiveness. Ensure the data distribution remains i.i.d., and the attacker's hyperparameters are unchanged from the template. Document and compare the performance of SparseFed to the simple clipping defense across these attack scenarios. Suggested values for k include percentages of the total parameters, such as 20%, 50%, and 80%.

- _(2 points)_ Analyze and discuss the results to determine if SparseFed provided any improvements. Explain why the sparsefication either succeeded or failed in enhancing the model's robustness against the specified attacks. Include this analysis in your notebook.

In [ ]:
def sparse_fed(
    all_updates: list[ClientUpdate], k: float, clip_norm_ratio: float = 1
) -> list[torch.Tensor]:
    clipped_parameters: list[torch.Tensor] = clipping(all_updates, clip_norm_ratio)

    def sparsify_parameter(parameter: torch.Tensor) -> torch.Tensor:
        parameter_shape = parameter.size()
        flattened_parameter = parameter.view(-1)

        retained_components_count = int(flattened_parameter.size(dim=0) * k)
        retained_components = torch.topk(flattened_parameter.abs(), k=retained_components_count)

        sparse_parameter = torch.zeros_like(flattened_parameter)
        sparse_parameter[retained_components.indices] = flattened_parameter[retained_components.indices]

        return sparse_parameter.view(*parameter_shape)

    return [sparsify_parameter(it) for it in clipped_parameters]

In [ ]:
def run_training(
    attack_type: typing.Literal["gradient-reversion", "backdoor"],
    defense_type: typing.Literal["clipping", "sparse-fed"],
    k: float,
) -> tuple[RunResult, BackdoorEvaluationResult]:
    dataset = split(nr_clients=100, iid=True, seed=42)

    match defense_type:
        case "clipping":
            defense = lambda all_updates: clipping(all_updates)
        case "sparse-fed":
            defense = lambda all_updates: sparse_fed(all_updates, k)

    server = configure_server(
        dataset,
        None,
        defense,
    )

    match attack_type:
        case "gradient-reversion":
            inject_malicious_clients(
                server, malicious_client_factory=build_gradient_reversion_client
            )
        case "backdoor":
            inject_malicious_clients(
                server, malicious_client_factory=build_backdoor_client
            )

    return server.run(10), evaluate_backdoor(server.model, test_loader)

In [ ]:
variants_sparse_fed = [0.2, 0.5, 0.8, 0.9, 0.95]

In [ ]:
result_clipping, _ = run_training('backdoor', 'clipping', 0)

In [ ]:
results_sparse_fed: list[RunResult] = []
results_backdoor_sparse_fed: list[BackdoorEvaluationResult] = []

for variant in tqdm(variants_sparse_fed, "variant", leave=False):
    result, backdoor_result = run_training('backdoor', 'sparse-fed', variant)

    results_sparse_fed.append(result)
    results_backdoor_sparse_fed.append(backdoor_result)

In [ ]:
result_clipping_reversion, _ = run_training('gradient-reversion', 'clipping', 0)

In [ ]:
results_sparse_fed_reversion: list[RunResult] = []

for variant in tqdm(variants_sparse_fed, "variant", leave=False):
    result, _ = run_training('gradient-reversion', 'sparse-fed', variant)

    results_sparse_fed_reversion.append(result)